In [1]:
import os
from copy import deepcopy
# import xml.etree.ElementTree as ET
# from rich.tree import Tree
# from rich import print as rprint
import io
from typing import List, Union, Tuple
# import pdfplumber
# import fitz 
import numpy as np
import pandas as pd
import requests
# import xmltodict
import re
from pypdf import PdfReader, PdfWriter
from difflib import SequenceMatcher
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.document import DocumentStream
from sentence_transformers import SentenceTransformer, util, CrossEncoder
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

/Users/justpqa/advpai/venv/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/Users/justpqa/advpai/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Try to look at the pdf

Try to look at the website of the paper instead

Try to use Docling

In [2]:
# first we need to know the number of col of a table to make sure we try the right orientation with docling
def extract_tables_num_col_lst_from_pmc(pmcid):
    url = f"https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{pmcid}/unicode"

    response = requests.get(url)
    tables_num_col_lst = []
    if response.status_code == 200:
        print(response.text)
        data = response.json()
        for d in data:
            doc = d["documents"]
            for p in doc:
                passage = p["passages"]
                for item in passage:
                    if item.get("infons", "").get("type", "").lower() == "table" and "text" in item:
                        table_str = item["text"]
                        num_col = 0
                        for row in table_str.split("\t \t"):
                            row_lst = row.split("\t")
                            num_col = max(num_col, len(row_lst))
                        tables_num_col_lst.append(num_col)
        print(f"Successfully retrieve number of columns")
    else:
        print(f"Failed to retrieve number of columns: {response.status_code}")
        
    return tables_num_col_lst

In [3]:
def clean_cell(val):
    tag_pattern = r'\s[a-z]$'
    if isinstance(val, str):
        return re.sub(tag_pattern, '', val)
    return val

def clean_headers(df):
    tag_pattern = r'\s[a-z]$'
    new_cols = []
    seen = {}
    for col in df.columns:
        if pd.isna(col):
            new_cols.append("")
        else:
            # 1. Apply the regex to the name string
            clean_name = re.sub(tag_pattern, '', str(col))
            # 2. Handle duplicates (e.g., if 'Price a' and 'Price b' both become 'Price')
            if clean_name in seen:
                seen[clean_name] += 1
                clean_name = f"{clean_name}_{seen[clean_name]}"
            else:
                seen[clean_name] = 0
            new_cols.append(clean_name)
    df.columns = new_cols
    return df

def extract_tables_lst_from_pdf_and_num_col(file_name, tables_num_col_lst):
    reader = PdfReader(file_name)
    options = PdfPipelineOptions()
    options.table_structure_options.mode = TableFormerMode.ACCURATE
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=options)
        }
    )
    df_lst = []
    row_pattern = r"\b(?:rs|s)\d+\S*"

    page_num = 1
    while len(df_lst) < len(tables_num_col_lst) and page_num <= len(reader.pages):
        for angle in [0, 90]: # Try normal, then try rotated
            writer = PdfWriter()
            page = reader.pages[page_num - 1]
            
            if angle != 0:
                page.rotate(angle)
            
            writer.add_page(page)
            
            # Convert just this one page
            pdf_buffer = io.BytesIO()
            writer.write(pdf_buffer)
            pdf_buffer.seek(0)
            
            doc_stream = DocumentStream(name=f"page_{page_num}.pdf", stream=pdf_buffer)
            result = converter.convert(doc_stream)
            
            # Check if this rotation produced valid table rows
            temp_dfs = []
            for table in result.document.tables:
                df = table.export_to_dataframe()
                # check if table is empty
                if (not df.empty):
                    # Case 1: continue from previous table
                    if len(df_lst) > 0 and df.shape[1] == df_lst[-1].shape[1]:
                        # extra filters for tables that are snp related, we need to remove rows that do not have snp id
                        # often are separation between sections
                        for col in df.columns:
                            # first modify "" -> nan
                            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True).ffill()
                        df["valid_row"] = df.apply(lambda x: any(re.search(row_pattern, str(c)) for c in x), axis=1)
                        df = df[df["valid_row"]].drop("valid_row", axis=1).reset_index().drop("index", axis = 1)
                        # some cell have the tag the end (often include a space and a small letter)
                        df = df.map(clean_cell) 
                        df = clean_headers(df) 
                        if df_lst[-1].columns.equals(df.columns):
                            df_lst[-1] = pd.concat([df_lst[-1], df], ignore_index = True)
                        elif df.shape[1] == tables_num_col_lst[len(df_lst) + len(temp_dfs)]:
                            # fail that test => add to temp since this is a new table
                            temp_dfs.append(df)
                    # Case 2: new table
                    elif (len(df_lst) + len(temp_dfs)) < len(tables_num_col_lst) and df.shape[1] == tables_num_col_lst[len(df_lst) + len(temp_dfs)]: 
                        # extra filters for tables that are snp related, we need to remove rows that do not have snp id
                        # often are separation between sections
                        for col in df.columns:
                            # first modify "" -> nan
                            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True).ffill()
                        df["valid_row"] = df.apply(lambda x: any(re.search(row_pattern, str(c)) for c in x), axis=1)
                        df = df[df["valid_row"]].drop("valid_row", axis=1).reset_index().drop("index", axis = 1)
                        # some cell have the tag the end (often include a space and a small letter)
                        df = df.map(clean_cell) 
                        df = clean_headers(df)             
                        temp_dfs.append(df)
            if temp_dfs:
                df_lst.extend(temp_dfs)
                break
            
        page_num += 1

    return df_lst

In [4]:
def extract_tables_lst_from_paper(pmcid, file_name, table_inx_to_extract=[]):
    tables_num_col_lst = extract_tables_num_col_lst_from_pmc(pmcid)
    if len(table_inx_to_extract) > 0:
        tables_num_col_lst = [tables_num_col_lst[i] for i in table_inx_to_extract]
    df_lst = extract_tables_lst_from_pdf_and_num_col(file_name, tables_num_col_lst)
    return df_lst

In [5]:
pmcid = "PMC10497850"
file_name = "papers/ACEL-22-e13938.pdf"
df_lst = extract_tables_lst_from_paper(pmcid, file_name)
for i in [3, 4]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

[{"source": "PMC", "date": "20231103", "key": "pmc.key", "infons": {}, "documents": [{"id": "10497850", "infons": {"license": "CC BY"}, "passages": [{"offset": 0, "infons": {"alt-title": "Dato et al.", "article-id_doi": "10.1111/acel.13938", "article-id_other": "ACE-23-0275.R1", "article-id_pmc": "10497850", "article-id_pmid": "37621137", "article-id_publisher-id": "ACEL13938", "elocation-id": "e13938", "fn": "Serena Dato and Francesco De Rango\u2014Co\u2010first authors. Giuseppina Rose and Valerio Napolioni\u2014Co\u2010last authors.", "issue": "9", "kwd": "Alzheimer's Disease epistasis gene\u2013gene interaction IGF1 longevity polymorphism", "license": "This is an open access article under the terms of the http://creativecommons.org/licenses/by/4.0/ License, which permits use, distribution and reproduction in any medium, provided the original work is properly cited.", "name_0": "surname:Dato;given-names:Serena", "name_1": "surname:De\u00a0Rango;given-names:Francesco", "name_2": "sur

Loading weights: 100%|██████████| 770/770 [00:00<00:00, 1486.31it/s, Materializing param=model.encoder_input_proj.2.1.weight]                                                 
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


In [6]:
pmcid = "PMC10115645"
file_name = "papers/41591_2023_Article_2268.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)
for i in [1, 2]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

[{"source": "PMC", "date": "20230422", "key": "pmc.key", "infons": {}, "documents": [{"id": "10115645", "infons": {"license": "CC BY"}, "passages": [{"offset": 0, "infons": {"article-id_doi": "10.1038/s41591-023-02268-w", "article-id_pmc": "10115645", "article-id_pmid": "37069360", "article-id_publisher-id": "2268", "fpage": "950", "issue": "4", "kwd": "Neurology Stroke Genome-wide association studies", "license": "Open Access This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons license, and indicate if changes were made. The images or other third party material in this article are included in the article\u2019s Creative Commons license, unless indicated otherwise in a credit line to the material. If material is not included in the article

Loading weights: 100%|██████████| 770/770 [00:00<00:00, 1610.42it/s, Materializing param=model.encoder_input_proj.2.1.weight]                                                 
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


In [7]:
pmcid = "PMC9622429"
file_name = "papers/nihms-1797266.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [3, 5, 6, 7, 8]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

[{"bioctype": "BioCCollection", "source": "PMC", "date": "20250420", "key": "pmc.key", "version": "1.0", "infons": {}, "documents": [{"bioctype": "BioCDocument", "id": "9622429", "infons": {"license": "author_manuscript"}, "passages": [{"bioctype": "BioCPassage", "offset": 0, "infons": {"article-id_doi": "10.1002/alz.12686", "article-id_manuscript": "NIHMS1797266", "article-id_pmc": "9622429", "article-id_pmid": "35490390", "fpage": "611", "issue": "2", "kwd": "cognitive impairment age at onset genome wide association study Cox mixed-effects models SHISA6 gene Alzheimer\u2019s disease protective variants", "license": "\n          This file is available for text mining. It may also be used consistent with the principles of fair use under the copyright law.\n        ", "lpage": "620", "name_0": "surname:Ramos;given-names:Jairo", "name_1": "surname:Caywood;given-names:Laura J.", "name_10": "surname:Main;given-names:Leighanne R.", "name_11": "surname:Osterman;given-names:Michael D.", "name

Loading weights: 100%|██████████| 770/770 [00:00<00:00, 1644.11it/s, Materializing param=model.encoder_input_proj.2.1.weight]                                                 
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


In [8]:
pmcid = "PMC10615750"
file_name = "papers/Recent paper on AD GWAS (1).pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [1, 2]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

[{"source": "PMC", "date": "20240127", "key": "pmc.key", "infons": {}, "documents": [{"id": "10615750", "infons": {"license": "CC BY"}, "passages": [{"offset": 0, "infons": {"article-id_doi": "10.1038/s41380-023-02089-w", "article-id_pmc": "10615750", "article-id_pmid": "37198259", "article-id_publisher-id": "2089", "fpage": "3121", "issue": "7", "kwd": "Neuroscience Genetics Diseases", "license": "Open Access This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons license, and indicate if changes were made. The images or other third party material in this article are included in the article\u2019s Creative Commons license, unless indicated otherwise in a credit line to the material. If material is not included in the article\u2019s Creative 

Loading weights: 100%|██████████| 770/770 [00:00<00:00, 1570.57it/s, Materializing param=model.encoder_input_proj.2.1.weight]                                                 
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


In [9]:
pmcid = "PMC6677735"
file_name = "papers/s42003-019-0537-9.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [1]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

[{"bioctype": "BioCCollection", "source": "PMC", "date": "20260202", "key": "pmc.key", "version": "1.0", "infons": {}, "documents": [{"bioctype": "BioCDocument", "id": "PMC6677735", "infons": {"license": "CC BY"}, "passages": [{"bioctype": "BioCPassage", "offset": 0, "infons": {"article-id_doi": "10.1038/s42003-019-0537-9", "article-id_pmc": "PMC6677735", "article-id_pmid": "31396565", "article-id_publisher-id": "537", "elocation-id": "285", "kwd": "Biomarkers Genome-wide association studies Neurology", "license": "Open Access This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons license, and indicate if changes were made. The images or other third party material in this article are included in the article\u2019s Creative Commons license, u

Loading weights: 100%|██████████| 770/770 [00:00<00:00, 1574.38it/s, Materializing param=model.encoder_input_proj.2.1.weight]                                                 
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.


In [10]:
pmcid = "PMC10286470"
file_name = "papers/s13024-023-00633-4.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [2, 3]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

[{"source": "PMC", "date": "20230704", "key": "pmc.key", "infons": {}, "documents": [{"id": "10286470", "infons": {"license": "CC BY"}, "passages": [{"offset": 0, "infons": {"article-id_doi": "10.1186/s13024-023-00633-4", "article-id_pmc": "10286470", "article-id_pmid": "37349795", "article-id_publisher-id": "633", "elocation-id": "40", "kwd": "Alzheimer\u2019s disease Genome-wide association study Cognitive domains Longitudinal measures Pleiotropy Pathway analysis", "license": "Open Access This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons licence, and indicate if changes were made. The images or other third party material in this article are included in the article's Creative Commons licence, unless indicated otherwise in a credit line

Loading weights: 100%|██████████| 770/770 [00:00<00:00, 1530.11it/s, Materializing param=model.encoder_input_proj.2.1.weight]                                                 
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_dataframe() without `doc` argument is deprecated.
Usage of TableItem.export_to_datafr

In [11]:
# pmcid = "PMC9945061"
# file_name = "PMID36809323_table_3.pdf"
# df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
# for i in [3]:
#     df_lst[i-3].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)
# problem in PMC api

Try to map columns to reference columns

In [12]:
# Now try to map the columns with the actual col in advp
referencing_cols = pd.read_csv("Rules for harmonizing ADVP papers - Main cols.csv")
referencing_cols["column_with_context"] = referencing_cols.apply(lambda x: x["column"] if pd.isna(x["description"]) else x["column"] + ": " + x["description"], axis = 1)
referencing_cols

,column,description,is_numeric,column_with_context
0,SNP,"Variant identifier, or snp idenifier, or chr:p...",False,"SNP: Variant identifier, or snp idenifier, or ..."
1,Chr,"Genomic chromosome identifier. Keywords: CHR, ...",False,Chr: Genomic chromosome identifier. Keywords: ...
2,Position,"Genomic coordinate location. Keywords: BP, POS...",False,Position: Genomic coordinate location. Keyword...
3,Locus,Specific named genomic region or hit identity....,False,Locus: Specific named genomic region or hit id...
4,Reported gene,Official gene symbol or name used in gene-base...,False,Reported gene: Official gene symbol or name us...
5,SNV,The count or specific list of variants in a ge...,False,SNV: The count or specific list of variants in...
6,Population,The ancestral group or ethnicity of the partic...,False,Population: The ancestral group or ethnicity o...
7,Cohort,The specific study or database name. Keywords:...,False,Cohort: The specific study or database name. K...
8,Sample size,Total number of subjects or observations. Keyw...,True,Sample size: Total number of subjects or obser...
9,Sample Information,Metadata regarding where sample data was extra...,False,Sample Information: Metadata regarding where s...


In [ ]:
# NOTE: before working in the main cell for mapping columns, write inference code here + setting up models here
def create_embeddings_from_model(sentences, model, tokenizer):
    input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # get token embeddings
    with torch.no_grad():
        output = model(**input)
    token_embeddings = output[0]

    # extract mask and mean pooling for sentence embeddings
    input_mask_expanded = input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
    sentence_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # final normalization
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

# def reranking_from_model(query, candidates, candidates_scores, batch_size = 16):
#     pairs = [[query, c] for c in candidates]
#     all_scores = []

#     with torch.no_grad():
#         for i in range(0, len(pairs), batch_size):
#             batch = pairs[i:i+batch_size]
#             inputs = reranker_tokenizer(
#                 batch,
#                 padding=True,
#                 truncation=True,
#                 max_length=512,
#                 return_tensors="pt",
#             )

#             logits = reranker_model(**inputs).logits.view(-1)
#             scores = torch.sigmoid(logits).detach().cpu().numpy()
#             all_scores.extend(scores.tolist())

#     best_idx = int(np.argmax(np.asarray(all_scores, dtype=float)))
#     return candidates[best_idx], candidates_scores[best_idx]

# def reranking_from_llm(query, query_details, candidates, candidates_details):
#     prompt = f"""
#     You are matching one source table column to the single best target schema field.

#     Rules:
#     - Pick exactly one candidate number that represents the same concept as the source column.
#     - Rely on meaning (units, value types, examples), not just wording.
#     - If no perfect match, choose the closest and mention uncertainty briefly.

#     Example:
#     Source column:
#     - name: "Position"
#     - details: Genomic coordinate location. Keywords: BP, POS, Base Pair, start, end. Examples: 45411941, 10240500:10248600 (range), build 37.

#     Candidates:
#     1. "Variant ID | description: dbSNP identifier 
#     2. "Chromosome position | description: genomic coordinate (bp)
#     3. "Effect allele | description: allele tested

#     Expected output:
#     BEST=2

#     Source column:
#     - name: {query}
#     - details: {query_details}

#     Candidates:
#     1. {candidates[0]} | descrption: {candidates_details[0]}
#     2. {candidates[1]} | descrption: {candidates_details[1]}
#     3. {candidates[2]} | descrption: {candidates_details[2]}
#     4. {candidates[3]} | descrption: {candidates_details[3]}
#     5. {candidates[4]} | descrption: {candidates_details[4]}

#     Output:
#     BEST=
#     """

#     inputs = llm_model_tokenizer(prompt, return_tensors="pt")

#     with torch.no_grad():
#         output_ids = llm_model.generate(
#             **inputs,
#             max_new_tokens=64,
#             do_sample=False,
#             temperature=0.0,
#         )

#     resp = llm_model_tokenizer.decode(output_ids[0], skip_special_tokens=True)
#     match = re.search(r"BEST\s*=\s*(\d+)", resp)
#     return candidates[int(match.group(1)) - 1] if int(match.group(1)) - 1 in range(len(candidates)) else query
    

# setting up models
embeddings_model_name = "NeuML/pubmedbert-base-embeddings"
embeddings_model_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_name)
embeddings_model = AutoModel.from_pretrained(embeddings_model_name)
embeddings_model.eval()

# setting up reranker
# reranker_model_name = "BAAI/bge-reranker-base"
# reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
# reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name)
# reranker_model.eval()

# setting up llm
# llm_model_name = "stanford-crfm/BioMedLM"
# llm_model_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
# llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name, torch_dtype=torch.float32).to("mps")
# llm_model.eval()

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 2024.74it/s, Materializing param=pooler.dense.weight]                               


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [14]:
# Try to first convert any abbreviation, then we match column with right semantic
gwas_abbreviation_dict = {
    "CHR": "Chromosome number",
    "BP": "Base-pair position",
    "POS": "Position",
    "SNP": "Single nucleotide polymorphism identifier",
    "RS": "Reference Single nucleotide polymorphism",
    "VAR": "Variant",
    "ID": "identifier",
    "A1": "Effect allele / tested allele",
    "A2": "Other allele / non-effect allele",
    "REF": "Reference allele (genome reference)",
    "ALT": "Alternate allele",
    "EA": "Effect Allele",
    "NEA": "Non-Effect Allele",
    "RA": "Risk Allele",
    "OA": "Other Allele",
    "AF": "Allele Frequency (general term)",
    "RAF": "Risk Allele Frequency",
    "EAF": "Effect Allele Frequency",
    "MAF": "Minor Allele Frequency",
    "BETA": "Effect size (regression coefficient)",
    "OR": "Odds Ratio",
    "SE": "Standard Error of effect estimate",
    "Z": "Z-score statistic",
    "T": "T-statistic",
    "CI": "Confidence Interval",
    "P": "P-value",
    "PVAL": "P-value",
    "LOGP": "Negative log10 P-value",
    "Q": "Heterogeneity statistic (meta-analysis)",
    "I2": "I-squared heterogeneity metric",
    "HWE": "Hardy-Weinberg Equilibrium test statistic",
    "INFO": "Imputation quality score",
    "R2": "Imputation accuracy metric",
    "CALLRATE": "Genotype call rate",
    "MISSING": "Missing genotype rate",
    "N": "Total sample size",
    "N_CASES": "Number of cases (for binary traits)",
    "N_CONTROLS": "Number of controls (for binary traits)",
    "EUR": "European ancestry",
    "AFR": "African ancestry",
    "ASN": "Asian ancestry",
    "AMR": "Admixed American ancestry",
    "SAS": "South Asian ancestry",
    "EAS": "East Asian ancestry",
    "LD": "Linkage Disequilibrium",
    "DPRIME": "LD D’ value",
    "CADD": "CADD score (functional impact)",
    "EQTL": "Expression quantitative trait locus",
    "PQTL": "Protein QTL",
    "GWGAS": "Gene-wide association study",
    "PRS": "Polygenic Risk Score",
    "PGS": "Polygenic Score",
    "QC": "Quality Control",
    "MA": "Meta-analysis",
    "HLA": "Human Leukocyte Antigen region",
    "HR": "Hazard ratio",
    "HET": "Heterogeneity test",
    "APOE4": "APOE ε4",
    "APOE*4": "APOE ε4",
    "#": "Number of",
    "frq": "Frequency",
    'β': "Effect",
    "nsnps": "Number of Variants"
}

In [15]:
def clean_col(col):
    """
    Clean a column by replacing any possible abbreviation with their actual meaning for better semantic matching
    """
    new_col = col
    for abb in gwas_abbreviation_dict:
        if re.search(fr"[^a-zA-Z]{abb.lower()}[^a-zA-Z]", new_col.lower()):
            new_col = re.sub(fr"([^a-zA-Z]){abb.lower()}([^a-zA-Z])", fr"\1{gwas_abbreviation_dict[abb]}\2", new_col.lower())
        elif re.search(fr"^{abb.lower()}[^a-zA-Z]", new_col.lower()):
            new_col = re.sub(fr"^{abb.lower()}([^a-zA-Z])", fr"{gwas_abbreviation_dict[abb]}\1", new_col.lower())
        elif re.search(fr"[^a-zA-Z]{abb.lower()}$", new_col.lower()):
            new_col = re.sub(fr"([^a-zA-Z]){abb.lower()}$", fr"\1{gwas_abbreviation_dict[abb]}", new_col.lower())
        elif re.search(fr"^{abb.lower()}$", new_col.lower()):
            new_col = re.sub(fr"{abb.lower()}", gwas_abbreviation_dict[abb], new_col.lower())
    # new_col = new_col.replace(".", " ")
    return new_col

In [16]:
def make_col_prompt(col, example_values, num_example_values = 5):
    """
    Based on the column title and some possible values of that columns, try to make a prompt
    """
    col_prompt = f"{col}: "
    for i in range(min(num_example_values, len(example_values))):
        col_prompt += f"{example_values[i]}, "
    return col_prompt

In [17]:
def match_single_col_to_ref_col(col, ref_col_lst, ref_col_embeddings):
    """
    match a single column to reference col given column, the list of referencing col and their embeddings
    """
    # calculate column embedding
    # col_embeddings = embeddings_model.encode(col, normalize_embeddings=True)
    # multi_index_pattern = r".+\..+"
    # if re.search(multi_index_pattern, col):
    #     col = col.replace(".", " ")
    #     # sub_col_lst = col.split(".")
    #     # sub_col_embeddings = create_embeddings_from_model(sub_col_lst, embeddings_model, embeddings_model_tokenizer)
    #     # col_embeddings = torch.mean(sub_col_embeddings, dim = 0)
    #     col = col.replace(".", " ")
    #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
    # else:
    #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
    col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
    col_embeddings = col_embeddings.reshape(-1, 1)

    # calculate similarity score
    scores = torch.matmul(ref_col_embeddings, col_embeddings).reshape(-1)

    # sort similairty score
    top_k_indices = torch.argsort(scores, descending=True)

    # verify if we even got good enough similarity
    best_inx = top_k_indices[0].item()
    best_score = scores[best_inx].item()
    # second_best_inx = top_k_indices[1].item()
    # second_best_score = scores[second_best_inx].item()
    # need a threshold for score or else, just return col
    # if best_score < 0.4: 
    #     return (col, 1)
    
    # # now do rerank
    # candidates = [ref_col_lst[inx] for inx in top_k_indices if scores[inx] >= 0.4]
    # candidates_scores = [scores[inx] for inx in top_k_indices if scores[inx] >= 0.4]
    # best_ref_col, best_ref_col_scores = reranking_from_model(col, candidates, candidates_scores)
    # return (best_ref_col, best_ref_col_scores)

    if best_score >= 0.4:
        return (ref_col_lst[best_inx], best_score)
    return (col, 1)

def match_many_col_to_ref_col(df, ref_col_df):
    """
    Given a list of column and a dataframe (could be dict later on if that fits better),
    try to match each column to the best fitted reference col, 
    return a dict of ref col : list of (col, cleaned col prompt, score)
    """
    # prepare the embeddings for reference col since we reuse them
    ref_col_lst = ref_col_df["column"].to_list()
    ref_col_context_lst = ref_col_df["column_with_context"].to_list()
    ref_col_embeddings = create_embeddings_from_model(ref_col_context_lst, embeddings_model, embeddings_model_tokenizer)

    # conduct matching
    multi_index_pattern = r"^.+\..+$" # need multi index pattern for handling multi index
    ref_col_to_col_lst = {}
    for col in df.columns:

        # extract values needed for prompt
        cleaned_col = clean_col(col)
        example_values = df[col].unique().tolist()
        # prompt: {col}: example, need to delete all : first

        if re.search(multi_index_pattern, cleaned_col.strip()):
            # try to assess each part and see if which one have highest score
            best_ref_col, best_score = None, 0
            best_cleaned_sub_col_prompt = None
            for sub_col in cleaned_col.split("."):

                # make column prompt for each sub col
                cleaned_sub_col_prompt = make_col_prompt(sub_col, example_values)
                
                # matching and compare
                ref_col, score = match_single_col_to_ref_col(cleaned_sub_col_prompt, ref_col_lst, ref_col_embeddings)
                if score > best_score and ref_col != cleaned_sub_col_prompt:
                    best_ref_col = ref_col
                    best_score = score
                    best_cleaned_sub_col_prompt = cleaned_sub_col_prompt
            
            # if we have a best one vs not
            if best_ref_col is not None:
                if best_ref_col not in ref_col_to_col_lst:
                    ref_col_to_col_lst[best_ref_col] = []
                ref_col_to_col_lst[best_ref_col].append((col, best_cleaned_sub_col_prompt, score))
            else:
                if col not in ref_col_to_col_lst:
                    ref_col_to_col_lst[col] = []
                ref_col_to_col_lst[col].append((col, best_cleaned_sub_col_prompt, 1))
        else:
            # match a single col with best fit referencing col and return a dict
            # make the col prompt
            # extra steps to remove .
            cleaned_col = cleaned_col.replace(".", "")
            cleaned_col_prompt = make_col_prompt(cleaned_col, example_values)

            # matching for best col
            ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
            # if we still get same col
            if ref_col == cleaned_col_prompt: 
                ref_col = col
            if ref_col not in ref_col_to_col_lst:
                ref_col_to_col_lst[ref_col] = []
            ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))
        # ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
        # if ref_col == cleaned_col_prompt: 
        #     ref_col = col
        # if ref_col not in ref_col_to_col_lst:
        #     ref_col_to_col_lst[ref_col] = []
        # ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

    return ref_col_to_col_lst

In [18]:
for file in os.listdir("./tables"):
    if "table" in file and ".csv" in file and "harmonized" not in file:
        print(file)
        df = pd.read_csv(f"./tables/{file}")
        df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
        col_to_ref_col = match_many_col_to_ref_col(df, referencing_cols)
        for ref_col in col_to_ref_col:
            print(f"{ref_col}: {col_to_ref_col[ref_col]}")
        print()

s42003-019-0537-9_table_1.csv
Lobe.: [('Lobe.', 'Lobe: Parietal, Temporal, Occipital, ', 1)]
Nearest gene: [('Annotation (nearby).', 'Annotation (nearby): UTR3, Intronic, Intergenic, Upstream, ', 0.6490411758422852)]
Position: [('Gene.', 'Gene: ZIC4, MSRB3, DAAM1 (dist = 24302), FGFRL1, CENPW (dist = 175626), ', 0.4209498167037964), ('Position.', 'Position: 147106319, 65793942, 59631075, 1013382, 126845380, ', 0.5123594403266907)]
SNP: [('Rs-ID.', 'reference single nucleotide polymorphism-identifier: rs2279829, rs146354218, rs147148763, rs74921869, rs1337736, ', 0.6220928430557251)]
Chr: [('Chr.', 'Chromosome number: 3, 12, 14, 4, 6, ', 0.6646981239318848)]
RA: [('A1.', 'Effect allele / tested allele: T, A, G, ', 0.6893497109413147), ('A2.', 'Other allele / non-effect allele: C, G, GTTGT, ', 0.5696174502372742)]
AF: [('European ancestry meta-analysis.Frq', 'european ancestry meta-analysis: 0.21, 0.37, 0.13, 0.2, 0.23, ', 0.4252067804336548), ('European ancestry meta-analysis.Effect', '

Final formatting of the tables

In [19]:
# We now can try to use these dictionary to make a final dataset
# need a list of columns that we sure that there might be multiple copies of it that we can melt them into many rows
possible_ref_col_to_melt = ["P-value", "Effect Size", "Confidence Interval", "AF"]
def format_original_table(df, remove_unique_col = False):
    # map the columns
    new_col_to_old_col_lst = match_many_col_to_ref_col(df, referencing_cols)
    df_with_ref_col = None
    new_col_to_not_melt = [] # list of columns that are stable and not need to be melt
    new_col_to_old_col_lst_to_melt = {}
    for new_col in new_col_to_old_col_lst:
        if len(new_col_to_old_col_lst[new_col]) == 1:
            if (not remove_unique_col) or (remove_unique_col and new_col_to_old_col_lst[new_col][0][2] != 1):
                if df_with_ref_col is None:
                    df_with_ref_col = df[[new_col_to_old_col_lst[new_col][0][0]]]
                    df_with_ref_col = df_with_ref_col.rename({new_col_to_old_col_lst[new_col][0][0]: new_col}, axis = 1)
                else:
                    df_with_ref_col[new_col] = df[new_col_to_old_col_lst[new_col][0][0]]
                # add these single col to the list of not melt
                new_col_to_not_melt.append(new_col)
        else:
            if new_col in possible_ref_col_to_melt:
                old_col_lst = []
                for col, _, _ in new_col_to_old_col_lst[new_col]:
                    old_col_lst.append(col)
                    if df_with_ref_col is None:
                        df_with_ref_col = df[[col]]
                    else:
                        df_with_ref_col[col] = df[col]
                new_col_to_old_col_lst_to_melt[new_col] = old_col_lst.copy()
            else:
                # make multiple copies with notes
                for inx, (col, _, _) in enumerate(new_col_to_old_col_lst[new_col]):
                    if df_with_ref_col is None:
                        df_with_ref_col = df[[col]]
                        df_with_ref_col = df_with_ref_col.rename({col: f"{new_col}_{inx + 1}"}, axis = 1)
                    else:
                        df_with_ref_col[f"{new_col}_{inx + 1}"] = df[col]
                    df_with_ref_col[f"{new_col}_{inx + 1} notes"] = col
                    # add these cols in group but not need to melt
                    new_col_to_not_melt.append(f"{new_col}_{inx + 1}")
                    new_col_to_not_melt.append(f"{new_col}_{inx + 1} notes")
    # Melting stage
    if len(new_col_to_old_col_lst_to_melt) > 0:
        # now melting column in same groups
        # Instead of keep melting, for each group, we make a new dataset of 
        # [stable col] + [to be melt col] => melt them as a new df
        # do this for each gorup and then join together based on stable col
        df_with_melt_col = None 
        for new_col in new_col_to_old_col_lst_to_melt:
            temp_df = deepcopy(df_with_ref_col[new_col_to_not_melt + new_col_to_old_col_lst_to_melt[new_col]])
            # create a temp row id for stable join
            temp_df["_row_id"] = np.arange(temp_df.shape[0])
            temp_df = temp_df.melt(
                id_vars = new_col_to_not_melt + ["_row_id"],    
                value_vars = new_col_to_old_col_lst_to_melt[new_col], 
                var_name = f"{new_col} notes",      
                value_name = f"{new_col}"
            )
            if df_with_melt_col is None:
                df_with_melt_col = deepcopy(temp_df)
            else:
                df_with_melt_col = df_with_melt_col.merge(temp_df, how = "inner", on = ["_row_id"] + new_col_to_not_melt)
        df_with_melt_col = df_with_melt_col.drop("_row_id", axis = 1)
        return df_with_melt_col
    else:
        return df_with_ref_col

In [20]:
modified_df_all = None
for file in os.listdir("./tables"):
    if ".csv" in file and "table" in file and "harmonized" not in file:
        df = pd.read_csv(f"./tables/{file}")
        df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
        modified_df = format_original_table(df, remove_unique_col = False)
        modified_df["file_name"] = file
        if modified_df_all is None:
            modified_df_all = modified_df.copy()
        else:
            modified_df_all = pd.concat([modified_df_all, modified_df], ignore_index = True)
        modified_df.to_csv(f"./harmonized_tables/{file.replace('.csv', '')}_harmonized.csv", index = False)
modified_df_all.to_csv("./harmonized_tables/harmonized_table.csv", index = False)